### Optimal Tactical Pathing for UGV

In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

In [3]:
m = gp.Model("mip1") 
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

Restricted license - for non-production use only - expires 2026-11-23


In [5]:
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
m.addConstr(x + y >= 1, "c1")

<gurobi.Constr *Awaiting Model Update*>

In [6]:
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
obj = gp.LinExpr()
obj += x
obj += y
obj += 2 * z
m.setObjective(obj, GRB.MAXIMIZE)

In [7]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 3 columns and 8 nonzeros
Model fingerprint: 0xae65aa22
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [8]:
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

x 1
y 0
z 1
